# Mapping Modelling Results

#### Author: alicefeng

This notebook provides a way to quickly visualize model-generated crash risk scores on a map.  Risk scores are color coded from yellow (lowest) to red (highest).

Inputs: 
- inter_and_non_int.shp - a shapefile that combines intersection and non-intersection segments with their ids as generated in create_segments.py
- a csv file that contains risk score predictions, where each row corresponds to one segment

Output: 
- a HTML file of a Leaflet map with risk scores by segment plotted on it

In [1]:
#Import the necessary Python modules
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm
#from folium.plugins import MarkerCluster
#from folium.element import IFrame

In [2]:
# Update below lines as needed to point to where you have stored your model output
output_fp = '../notebooks/benchmark/'
output_filename = 'seg_with_risk_score.csv'

In [3]:
# Read in model results
output = pd.read_csv(output_fp + output_filename)
output['id'] = output['segment_id'].astype('str')

In [4]:
# Read in shapefile as a GeoDataframe
streets = gpd.read_file('../data_generation/data/maps/inter_and_non_int.shp')

# Set the projection as EPSG:3857 since the shapefile didn't export with one
streets.crs = {'init': 'epsg:3857'}

# Then reproject to EPSG:4326 to match what Leaflet uses
streets = streets.to_crs({'init': 'epsg:4326'})

In [5]:
# Merge on model results to the GeoDataframe
streets_w_risk = streets.merge(output, on='id')

In [6]:
streets_w_risk[streets_w_risk.risk_score>0]

,geometry,id,segment_id,target,pre_week,pre_month,pre_quarter,avg_week,AADT,SPEEDLIMIT,Struct_Cnd,Surface_Tp,F_F_Class,risk_score
0,(LINESTRING (-71.13108292514419 42.23417947200...,10,10,0,0.0,0.0,0.0,0.020408,0,20,2,6,7,0.015610
4,(LINESTRING (-71.13190785549003 42.23720970210...,40,40,0,0.0,1.0,1.0,0.020408,20342,30,3,6,7,0.003906
13,(LINESTRING (-71.12243549173725 42.24467101882...,110,110,0,0.0,0.0,0.0,0.020408,16769,1,2,6,3,0.039062
20,(LINESTRING (-71.11377331324287 42.25258496788...,249,249,0,0.0,0.0,0.0,0.020408,0,20,2,6,7,0.015610
33,(LINESTRING (-71.11653205032023 42.25773060035...,335,335,0,0.0,0.0,0.0,0.020408,0,20,2,6,7,0.015610
37,(LINESTRING (-71.11638978719876 42.25989973954...,370,370,1,0.0,0.0,0.0,0.000000,3617,20,2,6,5,0.007812
39,(LINESTRING (-71.12817414771868 42.25804342703...,386,386,0,0.0,0.0,1.0,0.020408,9809,30,3,6,7,0.003906
41,(LINESTRING (-71.11717489103062 42.25820142596...,391,391,0,0.0,0.0,0.0,0.020408,0,20,2,6,7,0.015610
51,(LINESTRING (-71.10422010620042 42.26170979595...,450,450,0,0.0,0.0,0.0,0.000000,0,20,0,0,0,0.078125
54,(LINESTRING (-71.16249285662184 42.26242479095...,487,487,0,0.0,0.0,0.0,0.020408,0,20,2,6,7,0.015610


In [7]:
# Make map

# First create basemap
boston_map = folium.Map([42.3601, -71.0589], tiles='Cartodb Positron', zoom_start=12)  #"Cartodb dark_matter" also nice

# Create style function to color segments based on their risk score
color_scale = cm.linear.YlOrRd.scale(0, 1)
    
# Then add on GeoDataframe of risk scores
folium.GeoJson(streets_w_risk[streets_w_risk.risk_score>0],  # filter dataframe to only seg with risk>0 to reduce size
              style_function=lambda feature: {
                  'color': color_scale(feature['properties']['risk_score'])
              }).add_to(boston_map)

# Finally, add legend
color_scale.caption = "Risk Score"
boston_map.add_child(color_scale)

# Save map as separate html file
boston_map.save('risk_map.html')

boston_map